In [189]:
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# import nltk
import numpy as np
from math import ceil
from sklearn import svm, metrics


# nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words=set(stopwords.words('russian'))



file = open("news.txt")
full_text = ''
texts = []
labels = []
sentences = []


for item in file.readlines()[0:500]:
    # print(index)
    extracted_label = item.split('\t')[0]
    splitted_news_text = item.split('\t')[2]
    full_text = full_text + splitted_news_text
    
    cleared_text = ''
    for word in splitted_news_text.split(' '):
        lemmatizied_word = wordnet_lemmatizer.lemmatize(word)
        if (lemmatizied_word not in stop_words):
            cleared_text = cleared_text + word + ' '


    labels.append(extracted_label)
    texts.append(cleared_text)



int_labels = []
labels_set = list(set(labels))
for label in labels:
   for index, unique_label in enumerate(labels_set):
       if (label == unique_label):
           int_labels.append(index)
           break

print(labels_set, 'INITIAL LABELS')              
print(int_labels, 'NUMERICAL LABELS')       

data = pd.DataFrame(list(zip(texts))) 
data.columns = ['response']

response_new = data.response.apply(simple_preprocess)
# print(response_new)

# Model parameters
model=Word2Vec(window=25, min_count=2, workers=4, sg=1)

# Train the model
model.build_vocab(response_new, progress_per=1000)
model.train(response_new, total_examples=model.corpus_count, epochs=model.epochs)


# corpus_mean_vectors = []
# for corpus in response_new:
#     corpus_mean_vectors.append(model.wv.get_mean_vector(keys=corpus))


corpus_mean_vectors = []
for corpus in response_new:
    accumulative_vector = np.zeros(100)
    for word in corpus:
        word_distances = []
        best_vector = word
        # for wordJ in corpus:
        #     if (word in model.wv):
        #         word_vector = model.wv[word]
        #         # word_distances.append(model.wv.distance(word, wordJ))
        #     else:
        #         word_vector = np.zeros(100)

        if (word in model.wv):
            word_vector = model.wv[word]
        else:
            word_vector = np.zeros(100)
        for index, coord in enumerate(word_vector):
            accumulative_vector[index] = accumulative_vector[index] - coord #0.68
            # accumulative_vector[index] = accumulative_vector[index] + coord # 0.7
            # accumulative_vector[index] = min(accumulative_vector[index], coord) # 0.5
            # accumulative_vector[index] = max(accumulative_vector[index], coord) # 0.45
            # accumulative_vector[index] = accumulative_vector[index] + coord #0.56

    # for index, coord in enumerate(accumulative_vector):
    #     accumulative_vector[index] = coord / len(corpus)

    corpus_mean_vectors.append(accumulative_vector)


train_samples_size = ceil(len(corpus_mean_vectors) * 0.8)
X_train = corpus_mean_vectors[0: train_samples_size]
X_test = corpus_mean_vectors[train_samples_size:]
Y_train = int_labels[0: train_samples_size]
Y_test = int_labels[train_samples_size:]


clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(Y_pred)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
# Model Precision: what percentage of positive tuples are labeled as such?
# print("Precision:",metrics.precision_score(Y_test, Y_pred, average="weighted"))
# # Model Recall: what percentage of positive tuples are labelled as such?
# print("Recall:",metrics.recall_score(Y_test, Y_pred, average="weighted"))


['culture', 'forces', 'travel', 'business', 'style', 'economics', 'sport', 'life', 'science', 'media'] INITIAL LABELS
[4, 6, 9, 5, 5, 5, 0, 6, 9, 1, 9, 6, 1, 7, 1, 5, 0, 9, 9, 7, 9, 6, 5, 7, 0, 8, 8, 2, 6, 8, 5, 5, 9, 8, 5, 6, 1, 7, 9, 1, 0, 3, 9, 8, 5, 5, 1, 3, 6, 0, 8, 4, 0, 6, 9, 4, 0, 9, 7, 7, 6, 5, 6, 7, 7, 8, 5, 9, 8, 1, 8, 6, 0, 8, 8, 3, 8, 7, 1, 0, 9, 6, 0, 6, 1, 4, 7, 5, 8, 7, 9, 5, 0, 1, 6, 0, 8, 0, 8, 8, 8, 0, 0, 2, 2, 7, 1, 7, 9, 8, 8, 6, 7, 8, 8, 7, 8, 0, 9, 0, 1, 7, 0, 8, 7, 5, 5, 8, 7, 7, 6, 0, 9, 0, 3, 6, 9, 7, 9, 1, 9, 9, 5, 6, 5, 8, 3, 7, 8, 5, 7, 0, 0, 6, 8, 7, 7, 1, 5, 5, 5, 5, 6, 3, 0, 5, 6, 0, 4, 0, 8, 9, 1, 8, 4, 7, 8, 6, 0, 7, 9, 8, 8, 7, 7, 5, 1, 9, 9, 0, 0, 6, 3, 7, 0, 6, 8, 1, 5, 0, 0, 6, 6, 8, 9, 9, 5, 0, 5, 5, 0, 7, 8, 0, 0, 9, 9, 9, 1, 0, 3, 5, 0, 5, 8, 6, 0, 3, 0, 1, 5, 3, 5, 7, 9, 2, 7, 6, 5, 9, 3, 5, 8, 7, 7, 1, 5, 1, 8, 0, 0, 8, 9, 1, 5, 5, 7, 0, 9, 7, 0, 6, 6, 1, 7, 0, 6, 6, 8, 7, 8, 9, 9, 7, 7, 9, 0, 0, 9, 3, 9, 5, 9, 8, 6, 5, 7, 0, 6, 0, 5, 6, 9, 9,